<h1><center> Data Cleaning File  <br><br> 
<font color='grey'> Cleaning Emissions and Traffic Data <br><br>


Emissions Data

In [1]:
#setting libraries
import pandas as pd

In [2]:
#reading in emissions data files
emissions_16 = pd.read_excel("2016_emissions.xlsx",
                            sheet_name = "Direct Emitters", #only picks first sheet
                            header = 3,
                            usecols = ["City", "State", "Zip Code", "County", "Latitude", "Longitude", "Industry Type (subparts)", "Industry Type (sectors)", "Total reported direct emissions"])
emissions_13 = pd.read_excel("2013_emissions.xlsx",
                            sheet_name = "Direct Emitters", #only picks first sheet
                            header = 3,
                            usecols = ["City", "State", "Zip Code", "County", "Latitude", "Longitude", "Industry Type (subparts)", "Industry Type (sectors)", "Total reported direct emissions"])
emissions_19 = pd.read_excel("2019_emissions.xlsx",
                            sheet_name = "Direct Emitters", #only picks first sheet
                            header = 3,
                            usecols = ["City", "State", "Zip Code", "County", "Latitude", "Longitude", "Industry Type (subparts)", "Industry Type (sectors)", "Total reported direct emissions"])

In [3]:
#create column names
new_col_names = ["city", "state", "zipcode", "county", "latitude", "longitude", "industry_sub", "industry_main", "total_emissions"]

In [4]:
#rename columns
for df in [emissions_16, emissions_13, emissions_19]:
    df.columns = new_col_names

In [5]:
#save list of relevant states and industry
rel_states = ["DC", "MD", "VA"]
rel_indust = "MN|NN|C|W"

In [6]:
#filter for dc, md, and va + relevant industries
emissions_16 = emissions_16.query('state in @rel_states & industry_sub.str.contains(@rel_indust, na = False)')
emissions_13 = emissions_13.query('state in @rel_states & industry_sub.str.contains(@rel_indust, na = False)')
emissions_19 = emissions_19.query('state in @rel_states & industry_sub.str.contains(@rel_indust, na = False)')

In [7]:
#checking number of rows
[df.shape for df in [emissions_16, emissions_13, emissions_19]]

[(159, 9), (157, 9), (156, 9)]

In [8]:
#adding identifers before merging
emissions_13.loc[:, "time"] = 2013
emissions_16.loc[:, "time"] = 2016
emissions_19.loc[:, "time"] = 2019

C:\Users\helen\AppData\Local\Temp\ipykernel_9096\1078787289.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emissions_19.loc[:, "time"] = 2019


In [9]:
#merge
emissions = emissions_13.merge(emissions_16, how = "outer").merge(emissions_19, how = "outer")

In [10]:
#convert to wide format
emissions = emissions.pivot_table(
    index = list(emissions.columns[:-2]),
    columns = 'time',
    values = 'total_emissions'
).reset_index()

In [11]:
#set all colnames to str
emissions.columns = emissions.columns.astype(str)

In [12]:
#filters for all data that has both 2013 and 2019
emissions = emissions[(~emissions['2013'].isna()) & (~emissions['2019'].isna())]

In [13]:
emissions.shape

(92, 11)

In [14]:
emissions.query("state == 'DC'")

time,city,state,zipcode,county,latitude,longitude,industry_sub,industry_main,2013,2016,2019
210,WASHINGTON,DC,20052,DISTRICT OF COLUMBIA,38.897350,-77.045339,C,Other,22540.372,27369.360,34986.480
212,WASHINGTON,DC,20375,DISTRICT OF COLUMBIA,38.824587,-77.017398,C,Other,24441.600,21330.646,23672.632


In [15]:
emissions.query("state == 'MD'")

time,city,state,zipcode,county,latitude,longitude,industry_sub,industry_main,2013,2016,2019
11,Accident,MD,21520,GARRETT COUNTY,39.633667,-79.340479,"C,W-UNSTG",Petroleum and Natural Gas Systems,23114.338,20734.384,19969.488
12,BALTIMORE,MD,21202,BALTIMORE CITY,39.260810,-76.529950,C,Power Plants,35971.116,22582.014,17148.380
13,BALTIMORE,MD,21205,BALTIMORE CITY,39.298600,-76.563600,C,Power Plants,4788.324,7201.734,2318.410
14,BALTIMORE,MD,21218,BALTIMORE CITY,39.329070,-76.617910,C,Other,33655.124,30389.836,31195.482
19,BALTIMORE,MD,21226,BALTIMORE CITY,39.212440,-76.568160,C,Chemicals,119481.248,94151.242,115981.762
24,BALTIMORE,MD,21230,BALTIMORE CITY,39.280230,-76.571120,C,Other,101987.964,106311.298,105035.046
27,BETHESDA,MD,20892,MONTGOMERY COUNTY,39.003914,-77.099119,C,Other,243475.696,165911.554,217987.154
32,Baltimore,MD,21226,Anne Arundel,39.178100,-76.526800,"C,D",Power Plants,1466193.102,1184311.114,205671.188
33,Baltimore,MD,21226,Anne Arundel,39.179200,-76.538300,"C,D",Power Plants,4973556.612,5077126.318,2581355.832
60,CUMBERLAND,MD,21502,Allegany,39.594400,-78.745600,"C,D,PP","Power Plants,Suppliers of CO2",1482156.434,1091102.582,1226185.332


In [16]:
emissions.query("state == 'VA'")

time,city,state,zipcode,county,latitude,longitude,industry_sub,industry_main,2013,2016,2019
0,ACCOMAC,VA,23301,ACCOMACK COUNTY,37.736111,-75.656111,"C,II","Other,Waste",7.635612e+04,6.713106e+04,7.519194e+04
1,ALEXANDRIA,VA,22304,ALEXANDRIA CITY,38.801370,-77.128590,C,Waste,1.154345e+05,1.278904e+05,1.095711e+05
7,APPOMATTOX,VA,24522,APPOMATTOX COUNTY,37.345201,-78.873850,"C,W-NGTC",Petroleum and Natural Gas Systems,2.938478e+04,2.466460e+04,5.384419e+04
9,ASHLAND,VA,23005,HANOVER COUNTY,37.818100,-77.437500,C,Pulp and Paper,6.129559e+04,4.489044e+04,8.867872e+03
10,ASHLAND,VA,23005,Hanover,37.818100,-77.447800,"C,D",Power Plants,1.476661e+06,2.195375e+06,2.721285e+06
28,BIG ISLAND,VA,24526,Bedford,37.534000,-79.357000,"AA,C,TT","Pulp and Paper,Waste",2.036869e+05,2.143244e+05,2.432217e+05
29,BLACKSBURG,VA,24061,MONTGOMERY COUNTY,37.232150,-80.421730,C,Other,1.061052e+05,9.930704e+04,8.191927e+04
38,Bedford,VA,24523,BEDFORD COUNTY,37.256434,-79.482325,"C,HH",Waste,3.033020e+04,3.367090e+04,3.164160e+04
43,CHARLOTTESVILLE,VA,22903,CHARLOTTESVILLE CITY,38.035780,-78.514096,C,Other,9.749145e+04,8.554880e+04,8.331319e+04
44,CHATHAM,VA,24531,PITTSYLVANIA COUNTY,36.832360,-79.335870,"C,W-NGTC",Petroleum and Natural Gas Systems,1.150679e+04,NaN,1.280632e+05


Traffic Data

In [17]:
import requests
import os
import json

In [18]:
def traffic_query(coord_list):
    """
    takes in a list of captial bikeshare coordinates
    to tell API what data to retrieve
    """
    
    #extract coordinates
    long = coord_list[0] #UPDATE LATER AFTER SEEING HOW IBADAT SET IT UP
    lat = coord_list[1]
    
    #run query
    B_URL = "https://gis.mwcog.org/wa/rest/services/RTDC/Traffic_Counts_Annual/MapServer/0/query?" #base url
    
    response = requests.get(
        B_URL,
        params = {
            "where": "1=1", #no filters
            "outFields": "STATION,COUNTY,AADT2013,AADT2016, AADT2019,XCOORD,YCOORD", #indicates which cols to return
            "geometry": f"{long},{lat}", #input coordinates
            "geometryType": "esriGeometryPoint", #indicates we're giving it points
            "distance": 500, #how far away from point
            "units": "esriSRUnit_Meter", #units in meters
            "inSR": "4326", #coordiante system
            "f": "json" #type of file to return
        }
    )
    
    #check if successful
    if response.status_code == 200:
        return response.json()
    else:
        return "Query Failed"

In [19]:
def clean_traffic(geo_json, item_len):
    """
    function to extract columns of interest from traffic jsons
    takes in the json created in traffic_query() as well as the length
    of response.json()["features"]
    """
    #storing
    temp = []
    
    for item in range(item_len):
        obs = {
                "id": geo_json["features"][item]["attributes"]["STATION"],
                "x": geo_json["features"][item]["attributes"]["XCOORD"],
                "y": geo_json["features"][item]["attributes"]["YCOORD"],
                "county": geo_json["features"][item]["attributes"]["COUNTY"],
                "2013": geo_json["features"][item]["attributes"]["AADT2013"],
                "2016": geo_json["features"][item]["attributes"]["AADT2016"],
                "2019": geo_json["features"][item]["attributes"]["AADT2019"]
            }
        temp.append(obs)    
        
    #return 
    return(temp)
    

In [20]:
#test coordinates
long = -77.0234
lat = 38.89223

In [21]:
test_json = traffic_query([long, lat])

In [22]:
test_clean = clean_traffic(test_json, len(test_json["features"]))

In [23]:
pd.DataFrame(test_clean)

,id,x,y,county,2013,2016,2019
0,1313316,-77.020911,38.896131,District of Columbia,10688.0,11328.0,6995.0
1,7154,-77.028130,38.893552,District of Columbia,NaN,NaN,NaN
2,7153,-77.023972,38.892978,District of Columbia,NaN,NaN,NaN
3,7151,-77.021922,38.892686,District of Columbia,NaN,NaN,NaN
4,1010017,-77.026005,38.895236,District of Columbia,2769.0,2934.0,5765.0
5,7009,-77.021907,38.888150,District of Columbia,16717.0,19016.0,19450.0
6,3503,-77.027575,38.894878,District of Columbia,16881.0,23370.0,36107.0
7,3403,-77.019913,38.894004,District of Columbia,11006.0,12084.0,20676.0
8,1481617B,-77.019382,38.894579,District of Columbia,2860.0,3031.0,2056.0
9,1313316,-77.020911,38.896131,District of Columbia,NaN,NaN,NaN


In [24]:
temp = pd.DataFrame(test_clean)
temp[(~temp['2013'].isna()) & (~temp['2019'].isna())]

,id,x,y,county,2013,2016,2019
0,1313316,-77.020911,38.896131,District of Columbia,10688.0,11328.0,6995.0
4,1010017,-77.026005,38.895236,District of Columbia,2769.0,2934.0,5765.0
5,7009,-77.021907,38.888150,District of Columbia,16717.0,19016.0,19450.0
6,3503,-77.027575,38.894878,District of Columbia,16881.0,23370.0,36107.0
7,3403,-77.019913,38.894004,District of Columbia,11006.0,12084.0,20676.0
8,1481617B,-77.019382,38.894579,District of Columbia,2860.0,3031.0,2056.0
14,2105,-77.028051,38.890786,District of Columbia,11654.0,18375.0,28645.0
18,7154,-77.028130,38.893552,District of Columbia,18518.0,19625.0,21393.0
19,7153,-77.023972,38.892978,District of Columbia,19712.0,21480.0,27224.0
20,7151,-77.021922,38.892686,District of Columbia,15592.0,14325.0,19220.0
